# Generate a Project Template

## Imports

In [3]:
import subprocess
import json
import re
from pathlib import Path
from dataclasses import dataclass, InitVar

import pandas as pd

In [57]:
class ProjectException(Exception): 
    '''Base Exception for projects module.'''
    pass


class AnacondaException(ProjectException): 
    '''Errors related to `conda` calls.'''
    pass


## Utility Functions

In [33]:
@dataclass
class FolderItem:
    '''Manage package folders.'''
    folder_name: str
    tag: str = ''
    add_to_path: bool = True
    folder_path: Path = None
    base_path: InitVar[Path] = None

    def __post_init__(self, base_path: Path):
        '''If not defined, base_path becaomes the current working 
        directory.'''
        if not base_path:
            base_path = Path.cwd()
        self.set_path(base_path)
        self.set_tag()
            
    def set_path(self, base_path: Path):
        '''If missing, Set the folder_path to a subdirectory of base_path.

        Generates a path to the subdirectory.
        If it doesn't exist create it.

        Args:
            base_path (Path): The top folder 
                (parent to the subdirectory).
        '''
        if not self.folder_path:
            sub_folder = base_path / self.folder_name
            if not sub_folder.exists():
                sub_folder.mkdir()
            self.folder_path = sub_folder
            
    def set_tag(self):
        '''If tag not given use uppercase of folder name.'''
        if not self.tag:
            self.tag = self.folder_name.upper()

In [5]:
def add_folder(base: Path, name: str)->Path:
    '''Make a subdirectory under base.
    
    Generates a path to the subdirectory.
    If it doesn't exist create it.
    
    Args:
        base (Path): The top folder (parent to the subdirectory).
        name (str): The name of the subdirectory.

    Returns:
        Path: The path to the subdirectory.
    '''
    sub_folder = base / name
    if not sub_folder.exists():
        sub_folder.mkdir()
    return sub_folder


### Start Anaconda

In [44]:
anaconda_launcher = r'C:\ProgramData\Anaconda3\Scripts\activate.bat'
output = subprocess.run([anaconda_launcher], shell=True, capture_output=True)
type(output)

subprocess.CompletedProcess

In [58]:
def error_check(output: subprocess.CompletedProcess, error_type: Exception, 
                error_msg: str):
    '''Check for errors in subprocess output.
    
    if a `CalledProcessError` error occured raise the error, with the 
    error_msg text. 
    Args:
        output (subprocess.CompletedProcess): subprocess output to be tested.
        error_type (Exception): The type of exception to raise if an error 
            occurs.
        error_msg (str): The Error message to include if required.
    Raises:
        error_type: If subprocess generated an error. 
    '''
    try:
        output.check_returncode()
    except subprocess.CalledProcessError as err:
        msg = '\n'.join([error_msg, output.stderr.decode()])
        raise AnacondaException(msg) from err    

In [64]:
def console_command(cmd_str: str, error_type: Exception, error_msg: str)->str:
    '''Run a system console command.
    
    Run the command. Check for errors and raise the appropriate error if 
    nessesary.
    If no error, return the output from the result of running the console 
    command.

    Args:
        cmd_str (str): The console command to execute.
            command.
        error_type (Exception): The type of exception to raise if an error 
            occurs.
        error_msg (str): The Error message to include if required.

    Returns:
        str: The log output from running the console command.
    '''
    cmd_list = cmd_str.split(' ')
    output = subprocess.run(cmd_list, shell=True, capture_output=True)
    # check for errors
    error_check(output, error_msg)       
    cmd_log = output.stdout.decode()
    return cmd_log

In [65]:
def start_anaconda():
    anaconda_launcher = r'C:\ProgramData\Anaconda3\Scripts\activate.bat'
    output = console_command(r'C:\ProgramData\Anaconda3\Scripts\activate.bat', 
                             AnacondaException, 'Unable to start Anaconda')

In [66]:
start_anaconda()

In [67]:
def activate_environment(new_env):
    anaconda_activate = f'conda activate "{new_env}"'
    output = console_command(anaconda_activate, AnacondaException, 
                             f'Unable to activate environment {new_env}')

In [68]:
activate_environment(new_env)

AnacondaException: Unable to activate environment Todoist

EnvironmentNameNotFound: Could not find conda environment: "Todoist"
You can list all discoverable environments with `conda info --envs`.




### Install Jupyter Kernal 

In [43]:
def install_kernel(new_env):
    '''Install the ipython kernel in the new environment.

    Args:
        new_env (str): Name of Conda environment.

    Returns:
        str: The log output from the install.
    '''
    start_anaconda()
    install_kernel_cmd = ['python', '-m', 'ipykernel', 'install', 
                          '--user', '--name', new_env, 
                          '--display-name', f'"Python ({new_env})"']
    output = subprocess.run(install_kernel_cmd, shell=True, 
                            capture_output=True)
    # check for errors
    try:
        output.check_returncode()
    except subprocess.CalledProcessError as err:
        print(f'Unable to install ipython kernel in "{new_env}"!')
        print(output.stderr.decode())
        raise err
    else:   
        kernel_install_log = output.stdout.decode()
        return kernel_install_log

### Configure xlwings

In [63]:
start_anaconda()
activate_environment(new_env)
install_xlwings_cmd = ['xlwings', 'license', 'update', '-k', 'noncommercial']
output = subprocess.run(install_kernel_cmd, shell=True, capture_output=True)
# check for errors
error_check(output, f'Unable to install ipython kernel in "{new_env}"!')


In [32]:
def install_xlwings(new_env):
    '''Install xlwings binaries for environment.

    Args:
        new_env (str): Name of Conda environment.

    Returns:
        str: The log output from the install.
    '''
    !conda activate {new_env}
    license_log = !xlwings license update -k noncommercial
    adin_log = !xlwings addin install    
    config_log = !xlwings config create --force
    xlwing_log = '\n'.join([license_log, config_log, config_log])
    return xlwing_log

### Save Environment Specs

In [31]:
def save_env_specs(new_env: str, env_folder: Path):
    '''Store *spec* and *.yml* for the environment.

    Args:
        new_env (str): Name of Conda environment.
        env_folder (str): Path to the project's environment folder.
    '''
    spec_file = env_folder / f'{new_env}_specfile.txt'
    full_yml_file = env_folder / f'{new_env}_FULL.yml'
    history_yml_file = env_folder / f'{new_env}.yml'

    !conda activate "{new_env}"

    !conda list --explicit > "{str(spec_file)}"
    !conda env export > "{str(full_yml_file)}"
    !conda env export --from-history > "{str(history_yml_file)}"

## Project Parameters

### Project Name and Python Version

In [6]:
new_env = 'Todoist'
python_version = '3.11'

### Packages to include in the Conda Environment

In [7]:
primary_package_list = [
    'pysimplegui',
    'pandas',
    'xlwings',
    ]
tools_package_list = [
    'spyder',
    'jupyterlab',
    'ipykernel',
    'pylint',
    ]
package_list = primary_package_list + tools_package_list
packages = ' '.join(package_list)

In [8]:
install_xlwings = True
install_jupyter = True

## Specify project related paths
- The top level project path should already exist and should be given the tag 'WORKSPACE'
- `folder_path` must be specified for 'WORKSPACE'
- An 'ENVIRONMENT' must be defined.
- There shouldn't be duplicate tags.
- If `tag` is not given `folder_name.upper()` will be used as the `tag`
- When `folder_path` is given, the `folder_name` is ignored.  
- If `folder_path` is not given, a folder with the name `folder_name` will be created directly under `base_path`.  
- If `add_to_path` is True *(the default)*, the folder path will be added to the python search path.

#### Example folders:

| folder_name | folder_path                              | tag         | add_to_path |
|-------------|------------------------------------------|-------------|-------------|
| Todoist     | ~\Python\Projects\ToDoIst Projects\Turkey| WORKSPACE   | True        |
| environment | None                                     | ENVIRONMENT | False       |
| src         | None                                     | SOURCE      | True        |
| Old files   | None                                     | IGNORE      | False       |
| data        | None                                     | DATA        | True        |
| reference   | None                                     | REFERENCE   | False       |
| sectionary  | ..\\..\sectionary package\src'           | PACKAGE     | True        |

### Primary project path

In [9]:
project_folder = Path(r"D:\OneDrive - Queen's University\Python\Projects\ToDoIst Projects\Turkey")
project_drive = project_folder.drive

### Additional package path

In [10]:
sectionary_path = project_folder / '..' / '..' / 'sectionary package' / 'src'
sectionary_path = sectionary_path.resolve()

#### Folder Parameters

In [34]:
folder_list = [FolderItem(new_env,       tag='WORKSPACE',                      folder_path=project_folder),
               FolderItem('environment', tag='ENVIRONMENT', add_to_path=False,   base_path=project_folder),
               FolderItem('src',         tag='SOURCE',                           base_path=project_folder),
               FolderItem('data',        tag='DATA',                             base_path=project_folder),
               FolderItem('reference',   tag='REFERENCE',   add_to_path=False,   base_path=project_folder),
               FolderItem('sectionary',                                        folder_path=sectionary_path)]    

folders = pd.DataFrame(folder_list)


In [30]:
folders

,folder_name,tag,add_to_path,folder_path
0,Todoist,WORKSPACE,True,D:\OneDrive - Queen's University\Python\Projec...
1,environment,ENVIRONMENT,False,D:\OneDrive - Queen's University\Python\Projec...
2,src,SOURCE,True,D:\OneDrive - Queen's University\Python\Projec...
3,data,DATA,True,D:\OneDrive - Queen's University\Python\Projec...
4,reference,REFERENCE,False,D:\OneDrive - Queen's University\Python\Projec...
5,sectionary,SECTIONARY,True,D:\OneDrive - Queen's University\Python\Projec...


## Package Creation

### Create the new environment

In [9]:
print(f'Creating environment for {new_env}')
env_create_cmds = ['conda', 'create', '-y', '--name', new_env, f'python={python_version}', '--json']
output = subprocess.run(env_create_cmds,shell=True, capture_output=True)
# check for errors
try:
    output.check_returncode()
except subprocess.CalledProcessError as err:
    print(f'Unable to create new environment "{new_env}" with python={python_version}!')
    print(output.stderr.decode())
    raise err
    
install_log = output.stdout.decode()
#print(install_log)
logging_dict = json.loads(install_log)

Creating environment for Todoist


### Identify the Conda Environment path

In [13]:
env_path = Path(logging_dict['prefix'])

### Install the Packages

In [15]:
# Activate the new environment
env_activate_cmd = ['conda', 'activate', new_env]
output = subprocess.run(env_activate_cmd,shell=True, capture_output=True)
# check for errors
try:
    output.check_returncode()
except subprocess.CalledProcessError as err:
    print(f'Unable to activate new environment "{new_env}"!')
    print(output.stderr.decode())
    raise err

In [18]:
# Group Install the desired packages
env_install_cmds = ['conda', 'install', '-y', '--prefix', str(env_path), '--json'] + package_list
output = subprocess.run(env_install_cmds,shell=True, capture_output=True)
# check for errors
try:
    output.check_returncode()
except subprocess.CalledProcessError as err:
    print(f'Unable to install requested packages in "{new_env}"!')
    print(output.stderr.decode())
    raise err
else:   
    pkg_install_log = output.stdout.decode()
    pkg_install_dict = json.loads(pkg_install_log)


### Write the Log Data generated during the Conda Environment install

In [32]:

log_file = env_folder / f'{new_env}CreationLog.txt'
pkg_table_file = env_folder / f'{new_env}Packages.csv'

install_log = install_log + '\n' + pkg_install_log
log_file.write_text(install_log)

pkg_items = logging_dict['actions']['LINK'] + pkg_install_dict['actions']['LINK']
pkg_table = pd.DataFrame(pkg_items)
pkg_table.to_csv(pkg_table_file)

In [7]:
if install_jupyter:

In [33]:
if install_xlwings:
    

In [34]:
print(f'Storing environment for {new_env}')

## Build Boilerplate Files

### Build Jupyter lab Batch file

In [41]:

jupter_batch_file = env_folder / f'JupyterLab ({new_env}).bat'
jupyter_batch = '\n'.join([
    r'CALL C:\ProgramData\Anaconda3\Scripts\activate.bat C:\ProgramData\Anaconda3',
    f'CALL conda activate "{new_env}"',
    f'CD "{project_folder}"',
    f'{project_drive}',
    'jupyter-lab'
    ])
jupter_batch_file.write_text(jupyter_batch)

198

### Build VS Code Batch file

In [42]:
vscode_batch_file = env_folder / f'VS Code ({new_env}).bat'
vscode_batch = '\n'.join([
    r'CALL C:\ProgramData\Anaconda3\Scripts\activate.bat C:\ProgramData\Anaconda3',
    f'CALL conda activate "{new_env}"',
    f'CD "{project_folder}"',
    f'{project_drive}',
    f'code {new_env}.code-workspace'
    ])
vscode_batch_file.write_text(vscode_batch)

214

### Build .env file

In [14]:
folders

,folder_name,tag,add_to_path,folder_path
0,Todoist,WORKSPACE,True,D:\OneDrive - Queen's University\Python\Projec...
1,environment,ENVIRONMENT,False,D:\OneDrive - Queen's University\Python\Projec...
2,src,SOURCE,True,D:\OneDrive - Queen's University\Python\Projec...
3,data,DATA,True,D:\OneDrive - Queen's University\Python\Projec...
4,reference,REFERENCE,False,D:\OneDrive - Queen's University\Python\Projec...
5,sectionary,PACKAGE,True,D:\OneDrive - Queen's University\Python\Projec...


In [17]:
folders[folders.add_to_path]

,folder_name,tag,add_to_path,folder_path
0,Todoist,WORKSPACE,True,D:\OneDrive - Queen's University\Python\Projec...
2,src,SOURCE,True,D:\OneDrive - Queen's University\Python\Projec...
3,data,DATA,True,D:\OneDrive - Queen's University\Python\Projec...
5,sectionary,PACKAGE,True,D:\OneDrive - Queen's University\Python\Projec...


In [ ]:
folders[folders.tag=='ENVIRONMENT'].folder_path

In [51]:
env_file = env_folder / '.env'

env_paths = '\n'.join('PYTHONPATH=${}' for name, path_str in path_dict.items())
path_setter = '\n'.join('PYTHONPATH="${' + name.upper() + '};${PYTHONPATH}"' 
                        for name in path_dict.keys())
env_text = '\n'.join([env_paths, path_setter])
env_file.write_text(env_text)

183

### Build Code-workspace from templatestr(env_path)

In [15]:
data_dir = Path(r"D:\OneDrive - Queen's University\Python\Python Learning\Environment Management")

worspace_template = data_dir / 'vscode settings templates' / 'TEMPLATE.code-workspace'


In [16]:
worspace = worspace_template.read_text()

In [43]:
path_dict = {
    'WORKSPACE_FOLDER': f'"{project_folder}"',
    'SOURCE_FOLDER': f'"{source_folder}"',
    'SECTIONARY_FOLDER': f'"{sectionary_path}"'
    }

In [68]:
str(env_path).replace('\\', '/')

'C:/Users/smoke/.conda/envs/Todoist'

In [63]:
env_manage_base = Path(r"D:\OneDrive - Queen's University\Python\Python Learning\Environment Management")
code_workspace_template = env_manage_base / 'vscode settings templates' / 'TEMPLATE.code-workspace'

In [69]:
template = code_workspace_template.read_text()

In [70]:
template = template.replace('%env_path%', str(env_path).replace('\\', '/'))

In [35]:
project_folder = Path(r"D:\OneDrive - Queen's University\Python\Projects\ToDoIst Projects\Turkey")

env_folder = add_folder(project_folder, 'environment')
source_folder = add_folder(project_folder, 'src')

project_drive = project_folder.drive

In [43]:
sectionary_path = project_folder / '..' / '..' / 'sectionary package' / 'src'
sectionary_path = sectionary_path.resolve()

In [43]:
path_dict = {
    'WORKSPACE_FOLDER': f'"{project_folder}"',
    'SOURCE_FOLDER': f'"{source_folder}"',
    'SECTIONARY_FOLDER': f'"{sectionary_path}"'
    }

In [ ]:
template.replace('%analysis.extraPaths%', str(text))

In [ ]:
template.replace('%autoComplete.extraPaths%', str(text))

# Removing Environment

In [ ]:
#del_env = 'Standard'
#!conda activate {del_env}
#!xlwings addin uninstall
#!conda deactivate
#uninstall_log = !conda remove -y --name {del_env} --all
#uninstall_log

# Misc Reference